In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import os

import torch
from torchvision import models, transforms
from PIL import Image

resnet = models.resnet50(pretrained=True)
resnet.eval()

resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

In [ ]:
""""
256 - Either or both (the shortest of the two) the length or width is 256 pixels wide
224 - Center 224 x 224 pixels from image
Mean, std are obtained from the ImageNet dataset, used for training image classification models
    RGB order
"""
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        features = resnet(image)

    features = features.view(features.size(0), -1)
    return features
imagefolder = ""
feature_list = []
for skin_disease_folder, , files in os.walk(image_folder):
    for image in files:
        if image.endswith(".jpg"):
            image_path = os.path.join(skin_disease_folder, image)
            features = extract_features(image_path)
            feature_list.append(features)